In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from spellchecker import SpellChecker

app = Flask(__name__)

# ✅ Load dictionary safely
try:
    df = pd.read_csv("dict_cleaned.csv")
    if "word" not in df.columns:
        raise ValueError("CSV must have a column named 'word'")
    words = df["word"].dropna().astype(str).str.lower().unique()
except Exception as e:
    print("❌ Error loading dict_cleaned.csv:", e)
    words = []  # fallback to empty list

# ✅ Initialize SpellChecker with CSV words
spell = SpellChecker()
spell.word_frequency.load_words(words)

def autocorrect_text(text):
    corrected_words = []
    for w in text.split():
        word = w.lower()
        correction = spell.correction(word)
        corrected_words.append(correction if correction else word)
    return " ".join(corrected_words)

@app.route("/")
def home():
    return "Autocorrect API with dict.csv is running 🚀"

@app.route("/autocorrect", methods=["POST"])
def autocorrect():
    data = request.json
    text = data.get("text", "")
    return jsonify({
        "original": text,
        "corrected": autocorrect_text(text)
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


❌ Error loading dict.csv: [Errno 2] No such file or directory: 'dict.csv'
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.3.222:5000
Press CTRL+C to quit


In [2]:
pip install symspellpy

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\DELL\PycharmProjects\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from spellchecker import SpellChecker

app = Flask(__name__)

# ✅ Load dictionary
try:
    df = pd.read_csv("dict_cleaned.csv")
    if "word" not in df.columns:
        raise ValueError("CSV must have a column named 'word'")
    words = df["word"].dropna().astype(str).str.lower().unique()
except Exception as e:
    print("❌ Error loading dict.csv:", e)
    words = []

# ✅ Initialize SpellChecker
spell = SpellChecker()
spell.word_frequency.load_words(words)

def get_corrections(text):
    corrections = []
    corrected_words = []

    for w in text.split():
        lw = w.lower()
        if lw not in spell:
            suggestions = list(spell.candidates(lw))[:5]  # top 5
            best = spell.correction(lw)
            corrections.append({
                "word": w,
                "suggestions": suggestions
            })
            corrected_words.append(best if best else w)
        else:
            corrected_words.append(w)

    return {
        "original": text,
        "corrections": corrections,
        "corrected": " ".join(corrected_words)
    }

@app.route("/")
def home():
    return "Autocorrect API is running 🚀"

@app.route("/autocorrect", methods=["POST"])
def autocorrect():
    data = request.json
    text = data.get("text", "")
    return jsonify(get_corrections(text))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.3.222:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Sep/2025 10:15:32] "GET / HTTP/1.1" 200 -


In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv("dict.csv")

# 1. Remove rows where 'word' is missing
df = df.dropna(subset=['word'])

# 2. Drop duplicate rows
df = df.drop_duplicates()

# 3. Keep only the first unique definition per word
df = df.drop_duplicates(subset=['word'], keep='first')

# Save cleaned dataset
df.to_csv("dict_cleaned.csv", index=False)

print("Cleaning complete. Final shape:", df.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'dict.csv'